In [35]:
# Initial imports
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

#  Import and read the file.
import pandas as pd 
weather_df = pd.read_csv("./minute_weather.csv")
weather_df.head()

,rowID,hpwren_timestamp,air_pressure,air_temp,avg_wind_direction,avg_wind_speed,max_wind_direction,max_wind_speed,min_wind_direction,min_wind_speed,rain_accumulation,rain_duration,relative_humidity
0,0,9/10/2011 0:00,912.3,64.76,97.0,1.2,106.0,1.7,85.0,1.0,NaN,NaN,60.5
1,1,9/10/2011 0:01,912.3,63.86,161.0,0.8,215.0,1.5,43.0,0.2,0.0,0.0,39.9
2,2,9/10/2011 0:02,912.3,64.22,77.0,0.7,143.0,1.2,324.0,0.3,0.0,0.0,43.0
3,3,9/10/2011 0:03,912.3,64.40,89.0,1.2,112.0,1.6,12.0,0.7,0.0,0.0,49.5
4,4,9/10/2011 0:04,912.3,64.40,185.0,0.4,260.0,1.0,100.0,0.1,0.0,0.0,58.8


In [36]:
weather_df= weather_df.drop(['rowID', 'air_pressure', 'avg_wind_speed', 'max_wind_direction', 'max_wind_speed',	'min_wind_direction','min_wind_speed', 'rain_accumulation', 'rain_duration', 'relative_humidity', 'avg_wind_direction'],1)
weather_df.head()

,hpwren_timestamp,air_temp
0,9/10/2011 0:00,64.76
1,9/10/2011 0:01,63.86
2,9/10/2011 0:02,64.22
3,9/10/2011 0:03,64.40
4,9/10/2011 0:04,64.40


In [37]:
weather_df['hpwren_timestamp'] = pd.to_datetime(weather_df['hpwren_timestamp'])

In [38]:
def fun(datetimething):
     return datetimething.timetuple().tm_yday

In [39]:
weather_df['hpwren_timestamp'].apply(fun)

0          253
1          253
2          253
3          253
4          253
          ... 
1048570    228
1048571    228
1048572    228
1048573    228
1048574    228
Name: hpwren_timestamp, Length: 1048575, dtype: int64

In [40]:
# Create our features
X = weather_df.drop("hpwren_timestamp", axis = 1)
X = pd.get_dummies(X)

# Create our target
y = weather_df.loc[:, 'air_temp'].copy()

In [41]:
X.describe()

,air_temp
count,1.048575e+06
mean,6.088321e+01
std,1.202950e+01
min,3.164000e+01
25%,5.180000e+01
50%,6.080000e+01
75%,6.998000e+01
max,9.950000e+01


In [45]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [47]:
# Split our preprocessed data into our features and target arrays
y = weather_df.air_temp
X = weather_df.drop(columns="hpwren_timestamp")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [50]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [52]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units=80, activation="sigmoid", input_dim = number_input_features))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 80)                160       
                                                                 
 dense_3 (Dense)             (None, 1)                 81        
                                                                 
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________


In [53]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [54]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
24576/24576 [==============================] - 31s 1ms/step - loss: -912.4880 - accuracy: 0.0000e+00
Epoch 2/5
24576/24576 [==============================] - 37s 2ms/step - loss: -913.0766 - accuracy: 0.0000e+00
Epoch 3/5
24576/24576 [==============================] - 51s 2ms/step - loss: -913.0811 - accuracy: 0.0000e+00
Epoch 4/5
24576/24576 [==============================] - 33s 1ms/step - loss: -913.0793 - accuracy: 0.0000e+00
Epoch 5/5
24576/24576 [==============================] - 41s 2ms/step - loss: -913.0800 - accuracy: 0.0000e+00


In [55]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8192/8192 - 8s - loss: -9.1346e+02 - accuracy: 0.0000e+00 - 8s/epoch - 997us/step
Loss: -913.4575805664062, Accuracy: 0.0
